Another attempt to improve performance on diabetes dataset based on the feature interaction and feature importance insight gotten from analysis using the alibi package.

We will be dopping features "age" and "s6" due to low importance and minimal interaction with other features. This is is also intended to improve the signal to noise ratio of dataset.

We will be adding new interaction features namely: (s2*s5), (bp*s5), (s3*s5). 

K-fold cross-validation will be maintained for a more realistic performance insight.

At end, model peformances will be compared with performances from cross_validation alone. Also SNR will be computed for new improved dataset.

Note: Interaction and importance ing=sights were computed using MLP regressor as model.

Performance metrics: MSE, R2 score.

In [1]:
from sklearn.datasets import load_iris, load_diabetes
from sklearn.model_selection import KFold, cross_val_score
import pandas as pd
import numpy as np

In [2]:
diabetesData = load_diabetes()  #load diabetes dataset to the diabetesData variable
for key in diabetesData.keys(): #Assessment of keys of dataset
    print(key, end="\n")

# print(diabetesData.DESCR, end="\n")
#Constructing the data array(predictor) into a pandas dataframe
X = pd.DataFrame(diabetesData.data, columns=diabetesData.feature_names)
X

data
target
frame
DESCR
feature_names
data_filename
target_filename
data_module


,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641
...,...,...,...,...,...,...,...,...,...,...
437,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207
438,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018114,0.044485
439,0.041708,0.050680,-0.015906,0.017293,-0.037344,-0.013840,-0.024993,-0.011080,-0.046883,0.015491
440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044529,-0.025930


In [3]:
X["diseaseProg"] = diabetesData.target  #Add the taget column to feature dataset to check for feature correlations, especially with target.
X.corr()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,diseaseProg
age,1.000000,0.173737,0.185085,0.335428,0.260061,0.219243,-0.075181,0.203841,0.270774,0.301731,0.187889
sex,0.173737,1.000000,0.088161,0.241010,0.035277,0.142637,-0.379090,0.332115,0.149916,0.208133,0.043062
bmi,0.185085,0.088161,1.000000,0.395411,0.249777,0.261170,-0.366811,0.413807,0.446157,0.388680,0.586450
bp,0.335428,0.241010,0.395411,1.000000,0.242464,0.185548,-0.178762,0.257650,0.393480,0.390430,0.441482
s1,0.260061,0.035277,0.249777,0.242464,1.000000,0.896663,0.051519,0.542207,0.515503,0.325717,0.212022
s2,0.219243,0.142637,0.261170,0.185548,0.896663,1.000000,-0.196455,0.659817,0.318357,0.290600,0.174054
s3,-0.075181,-0.379090,-0.366811,-0.178762,0.051519,-0.196455,1.000000,-0.738493,-0.398577,-0.273697,-0.394789
s4,0.203841,0.332115,0.413807,0.257650,0.542207,0.659817,-0.738493,1.000000,0.617859,0.417212,0.430453
s5,0.270774,0.149916,0.446157,0.393480,0.515503,0.318357,-0.398577,0.617859,1.000000,0.464669,0.565883
s6,0.301731,0.208133,0.388680,0.390430,0.325717,0.290600,-0.273697,0.417212,0.464669,1.000000,0.382483


In [4]:
for featureName in X.columns:
    if featureName == "diseaseProg":
            X = X.drop("diseaseProg", axis=1)   #drop the target column after adding it to check for feature corelations.
#Constructing the target array(response/target) into a pandas dataframe
y = pd.DataFrame(diabetesData.target, columns=["DiseaseProgression"])
X

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641
...,...,...,...,...,...,...,...,...,...,...
437,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207
438,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018114,0.044485
439,0.041708,0.050680,-0.015906,0.017293,-0.037344,-0.013840,-0.024993,-0.011080,-0.046883,0.015491
440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044529,-0.025930


In [5]:
# #Splitting the dataset
# from sklearn.model_selection import train_test_split as ttsplit

# XTrain, XTest, yTrain, yTest = ttsplit(diabetesPredictorDataFrame, diabetesTargetDataFrame)

#Here, we import our metrics:
from sklearn.metrics import mean_squared_error, r2_score

#Next, we create numpy 2D array to hold our calculated mse and r2 score metrics for 
# the train and test sets. We wil use it later to visualize their performances tabularly.
rowsOfMetrics = 2   
noOfModelsEvaluated = 6
metrics = np.zeros((rowsOfMetrics, noOfModelsEvaluated))


In [6]:
#Here we import and train and evaluate a Linear Regression model.
from sklearn.linear_model import LinearRegression

lrModel = LinearRegression()

lrKf = KFold(n_splits=2)

lrCvMseScore = cross_val_score(estimator=lrModel, X=X, y=y, cv=lrKf, scoring="neg_mean_squared_error")
lrCvR2Score = cross_val_score(estimator=lrModel, X=X, y=y, cv=lrKf, scoring="r2")

#Model evaluation and assignment of metrics to respective indices in metrics array
metrics[0, 0] = np.mean(abs(lrCvMseScore))
metrics[1, 0] = np.mean(lrCvR2Score)

In [7]:

#Here we train and evaluate a Lasso Regession Model:
from sklearn.linear_model import Lasso

lassoModel = Lasso(alpha=0.01) #regularization strenth of 1.
lassoKf = KFold(n_splits=10)

lassoCvMseScore = cross_val_score(lassoModel, X, y, cv=lassoKf, scoring="neg_mean_squared_error")
lassoCvR2Score = cross_val_score(estimator=lassoModel, X=X, y=y, cv=lassoKf, scoring="r2")

#Model evaluation and assignment of metrics to respective indices in metrics array
metrics[0, 1] = np.mean(abs(lassoCvMseScore))
metrics[1, 1] = np.mean(lassoCvR2Score)

In [8]:
# %%script false

#Here we train and evaluate a Ridge Regession Model:
from sklearn.linear_model import Ridge

ridgeModel = Ridge(alpha= 0.05)
ridgeKf = KFold(n_splits=10)

ridgeCvMseScore = cross_val_score(estimator=ridgeModel, X=X, y=y, cv=ridgeKf, scoring="neg_mean_squared_error")
ridgeCvRScore = cross_val_score(estimator=ridgeModel, X=X, y=y, cv=ridgeKf, scoring="r2")

#Model evaluation and assignment of metrics to respective indices in metrics array
metrics[0, 2] = np.mean(abs(ridgeCvMseScore))
metrics[1, 2] = np.mean(ridgeCvRScore)


In [9]:
# %%script false
#Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor

rfRegressor = RandomForestRegressor(100, criterion='absolute_error', max_depth=4, random_state=42)
rfKfold = KFold(n_splits=10)

rfCvMseScore = cross_val_score(estimator=rfRegressor, X=X, y=y, cv=ridgeKf, scoring="neg_mean_squared_error")
rfCvR2Score = cross_val_score(estimator=rfRegressor, X=X, y=y, cv=ridgeKf, scoring="r2")

#Model evaluation and assignment of metrics to respective indices in metrics array
metrics[0, 3] = np.mean(abs(rfCvMseScore))
metrics[1, 3] = np.mean(rfCvR2Score)


c:\Users\DELL\AppData\Local\Programs\Python\Python314\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\DELL\AppData\Local\Programs\Python\Python314\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\DELL\AppData\Local\Programs\Python\Python314\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\DELL\AppData\Local\Programs\Python\Python314\Lib\site-packages\sklearn\base.py:1365: DataConversionW

In [10]:
# %%script false

#Training of an XGBoost model, to check if it gives a yet better peformance.
from xgboost import XGBRegressor
xgbModel = XGBRegressor(device="cpu",max_depth=4,random_state=42)
xgbKf = KFold(n_splits=10)

xgbCvMseScore = cross_val_score(estimator=xgbModel, X=X, y=y, cv=ridgeKf, scoring="neg_mean_squared_error")
xgbCvR2Score = cross_val_score(estimator=xgbModel, X=X, y=y, cv=ridgeKf, scoring="r2")

#Model evaluation and assignment of metrics to respective indices in metrics array
metrics[0, 4] = np.mean(abs(xgbCvMseScore))
metrics[1, 4] = np.mean(xgbCvR2Score)

In [11]:
from sklearn.neural_network import MLPRegressor

nnModel = MLPRegressor()
nnKf = KFold(n_splits=10)

nnCvMseScore = cross_val_score(estimator=nnModel, X=X, y=y, cv=nnKf, scoring="neg_mean_squared_error")
nnCvR2Score = cross_val_score(estimator=nnModel, X=X, y=y, cv=nnKf, scoring="r2")

#Model evaluation and assignment of metrics to respective indices in metrics array
metrics[0, 5] = np.mean(abs(nnCvMseScore))
metrics[1, 5] = np.mean(nnCvR2Score)

c:\Users\DELL\AppData\Local\Programs\Python\Python314\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1771: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\DELL\AppData\Local\Programs\Python\Python314\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\DELL\AppData\Local\Programs\Python\Python314\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1771: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\DELL\AppData\Local\Programs\Python\Python314\Lib\site-packages\sklearn\neural_network\_multilayer_

In [12]:
# %%script false

#Tabular visualization of performace metrics of models on trining and test data subsets.
index = ["MSE score", "R2 score"]
columns = ["OLSRegression", "LassoRegression", "RidgeRegression", "RandomForest", "XGBoost", "NeuralNetwok"]
metrics = pd.DataFrame(metrics, index=index, columns=columns)
metrics

,OLSRegression,LassoRegression,RidgeRegression,RandomForest,XGBoost,NeuralNetwok
MSE score,2992.335630,3004.095720,2997.229190,3269.982650,4195.118823,24079.673222
R2 score,0.492439,0.461785,0.463474,0.414577,0.252512,-3.279415


In [13]:
# Calculating Signal to noise ratio of dataset: 1st approach
# snr = np.mean(y, axis=0)/np.var(y, axis=0)
# print(f"Signal to noise ratio = {snr}")

# 2nd approach:
#I'll use OLS regression as our estimator/model

from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.metrics import r2_score

rkf = RepeatedKFold(n_splits=5, n_repeats=5)

cvTargetR2 = cross_val_score(lrModel, X, y, cv=rkf, scoring="r2")   #LinearRegression() rep by lrModel is used to compute SNR because it is the best performing model yet.

r2Score = cvTargetR2.mean()
#Recall, signal = r2 score. And noise = (1 - r2)
#Therefore, SNR = r2/(1 - r2)

snr = r2Score/(1 - r2Score)
print(f"Signal to noise ratio of dataset = {snr}") 

Signal to noise ratio of dataset = 0.9180969991984741


Notes: 
1. As the alpha (regularization strength) value increases, the regularization increases causing the regularizeed models to underfit, performing more poorly than OLS.
2. MSE and r2 scores appear terible mainly due to smallness of dataset size.
3. I tried K-Fold cross-validation here, checking if it yields better performance metrics. But it appears even worse. But this mirrors more accurately model's performance regardless.